In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import requests
import json
from config import api_key
print(api_key)

906d6b2110ef41ccb46203211180711


In [11]:
# Save config information.
#variables to append queried API data for each zipcodes
longt = []
lat = []
temp = []
humidity = []
cloudiness = []
wind_speed = []
city_name = []
country_name = []
date = []
max_temp = []

#http://api.apixu.com/v1/search.json?key=<YOUR_API_KEY>&q=lond
historic_url = "http://api.apixu.com/v1/history.json?"
forecast_url = "http://api.apixu.com/v1/forecast.json?"
current_url = "http://api.apixu.com/v1/current.json?"
#units = "imperial"
# Build partial query URL
current_query_url = f"{current_url}key={api_key}&q="
#historical request, dt should be in yyyy-MM-dd format after 1st Jan, 2015
date
historic_query_url = f"{historic_url}key={api_key}&q="
response = requests.get(f"{historic_url}&q='91913'&dt=2018-11-05").json()

response
# print("Begining Data Retrieval") 
# print("------------------------") 
# for zcode in zipcodes:
#     try:
#         response = requests.get(query_url + zcode).json()
#         lat.append(response['coord']['lat'])
#         longt.append(response['coord']['lon'])
#         temp.append(response['main']['temp'])
#         humidity.append(response['main']['humidity'])
#         cloudiness.append(response['clouds']['all'])
#         wind_speed.append(response['wind']['speed'])
#         city_name.append(response['name'])
#         country_name.append(response['sys']['country'])
#         date.append(time.ctime(response['dt']))
#         max_temp.append(response['main']['temp_max'])
#         print(f"Retrieving Results for Index| {cities.index(zcode)}| {zcode}.")
#         print(f"{query_url + zcode}")
        
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")

        
# print("--------------------------------------------------")
# Print("Data Retrieval Complete")

{'location': {'name': 'Chula Vista',
  'region': 'California',
  'country': 'USA United States of America',
  'lat': 32.64,
  'lon': -116.98,
  'tz_id': 'America/Los_Angeles',
  'localtime_epoch': 1541804700,
  'localtime': '2018-11-09 15:05'},
 'forecast': {'forecastday': [{'date': '2018-11-05',
    'date_epoch': 1541376000,
    'day': {'maxtemp_c': 22.2,
     'maxtemp_f': 72.0,
     'mintemp_c': 16.2,
     'mintemp_f': 61.2,
     'avgtemp_c': 19.2,
     'avgtemp_f': 66.6,
     'maxwind_mph': 6.9,
     'maxwind_kph': 11.2,
     'totalprecip_mm': 0.0,
     'totalprecip_in': 0.0,
     'avgvis_km': 10.0,
     'avgvis_miles': 6.0,
     'avghumidity': 64.0,
     'condition': {'text': 'Sunny',
      'icon': '//cdn.apixu.com/weather/64x64/day/113.png',
      'code': 1000},
     'uv': 0.0},
    'astro': {'sunrise': '06:09 AM',
     'sunset': '04:54 PM',
     'moonrise': '03:59 AM',
     'moonset': '04:07 PM',
     'moon_phase': 'Waning Crescent',
     'moon_illumination': '21'},
    'hour': [

Current weather object for Paris: {'coord': {'lon': 2.35, 'lat': 48.86}, 'weather': [{'id': 701, 'main': 'Mist', 'description': 'mist', 'icon': '50n'}], 'base': 'stations', 'main': {'temp': 6.37, 'pressure': 1019, 'humidity': 87, 'temp_min': 5, 'temp_max': 7}, 'visibility': 10000, 'wind': {'speed': 3.1, 'deg': 160}, 'clouds': {'all': 20}, 'dt': 1541649600, 'sys': {'type': 1, 'id': 5610, 'message': 0.0054, 'country': 'FR', 'sunrise': 1541659718, 'sunset': 1541693963}, 'id': 2988507, 'name': 'Paris', 'cod': 200}.
